<a href="https://colab.research.google.com/github/Elshamysamira/Machine-Learning-Methods/blob/nasti/QA_inverted_idx_Bublin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk

!pip install datasets
import nltk

nltk.download('punkt')  # This downloads necessary datasets for tokenization

from nltk.tokenize import word_tokenize
from collections import defaultdict

import pandas as pd

from pathlib import Path
import os
import sqlite3

from collections import defaultdict
import re
from datasets import load_dataset


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
cuad_dataset = load_dataset("cuad")

# Select only 20 examples from the dataset
cuad_dataset = cuad_dataset['train'].select(range(20))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/22450 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4182 [00:00<?, ? examples/s]

In [3]:
class InvertedIndex:
    def __init__(self):
        self.inverted_index = defaultdict(set)

    def build_index(self, documents):
        for documentID, document_content in enumerate(documents):
            for word in document_content:
                self.inverted_index[word].add(documentID)

    def save_index(self, db_path):
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute('''CREATE TABLE IF NOT EXISTS inverted_index (
                                word TEXT PRIMARY KEY,
                                document_ids TEXT
                            )''')
        for word, document_ids in self.inverted_index.items():
            doc_ids_str = ','.join(str(doc_id) for doc_id in document_ids)
            cursor.execute("INSERT INTO inverted_index (word, document_ids) VALUES (?, ?)", (word, doc_ids_str))
        conn.commit()
        conn.close()

In [4]:
class SearchEngine:
    def __init__(self, index_db_path, documents):
        self.index_db_path = index_db_path
        self.documents = documents

    def lookup_word(self, word):
        conn = sqlite3.connect(self.index_db_path)
        cursor = conn.cursor()
        cursor.execute("SELECT document_ids FROM inverted_index WHERE word=?", (word,))
        result = cursor.fetchone()
        conn.close()

        if result:
            document_ids = set(map(int, result[0].split(',')))
            return document_ids
        else:
            return set()

    def search(self, query):
        # Tokenize the query
        query_tokens = word_tokenize(query.lower())
        print(f"Tokenized Query: {query_tokens}")  # Print tokenized query

        # Look up each token in the inverted index
        document_sets = [self.lookup_word(token) for token in query_tokens]

        # Find common documents where all words appear
        common_documents = set.intersection(*document_sets) if document_sets else set()

        if common_documents:
            print(f"Congratulations! The word(s) '{query}' appear together in the following document ID(s): {common_documents}")
            for doc_id in common_documents:
                print(f"Document ID: {doc_id}, Document Name: {self.documents[doc_id]}")
        else:
            print(f"I'm sorry, the word(s) '{query}' do not appear together in any document.")


In [5]:
# Tokenize documents
tokenized_docs = [word_tokenize(example['context'].lower()) for example in cuad_dataset]

# Build inverted index
index = InvertedIndex()
index.build_index(tokenized_docs)

# Save inverted index to SQLite
index.save_index('cuad_inverted_index.db')

# Document mapping
document_mapping = {i: f"CUAD Document {i}" for i in range(len(tokenized_docs))}

# Initialize SearchEngine
search_engine = SearchEngine('cuad_inverted_index.db', document_mapping)


In [6]:
# Search
search_engine.search('confidential')
print('\n')
search_engine.search('Efficiency')
print('\n')
search_engine.search('consequence of Force Majeure')


Tokenized Query: ['confidential']
Congratulations! The word(s) 'confidential' appear together in the following document ID(s): {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
Document ID: 0, Document Name: CUAD Document 0
Document ID: 1, Document Name: CUAD Document 1
Document ID: 2, Document Name: CUAD Document 2
Document ID: 3, Document Name: CUAD Document 3
Document ID: 4, Document Name: CUAD Document 4
Document ID: 5, Document Name: CUAD Document 5
Document ID: 6, Document Name: CUAD Document 6
Document ID: 7, Document Name: CUAD Document 7
Document ID: 8, Document Name: CUAD Document 8
Document ID: 9, Document Name: CUAD Document 9
Document ID: 10, Document Name: CUAD Document 10
Document ID: 11, Document Name: CUAD Document 11
Document ID: 12, Document Name: CUAD Document 12
Document ID: 13, Document Name: CUAD Document 13
Document ID: 14, Document Name: CUAD Document 14
Document ID: 15, Document Name: CUAD Document 15
Document ID: 16, Document Name: CUAD D

In [7]:
search_engine.search('AbrakadaBbRA')

Tokenized Query: ['abrakadabbra']
I'm sorry, the word(s) 'AbrakadaBbRA' do not appear together in any document.
